In [103]:
# import random
# import operator
import numpy as np
# import pandas as pd
# # from binarytree import Node,build
# import random
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
import operator

import deap
from deap import base
from deap import creator
from deap.gp import *
from deap.tools import *





TEMPERATURE = 1
P_CORRELATION_BOUND = .7

Pc = .95
Pm = .05

NO_OF_GENERATIONS = 100

N_ROWS = 10
N_COLUMNS = 10

N_ROWS = 10
N_COLUMNS = 10
P_CORRELATION_BOUND = .7   #TODO: RELATE WITH TEMPERATURE


def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

    
# GP Expression Trees
pset = PrimitiveSet("main", 9)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
# pset.addEphemeralConstant("randconst", lambda: random.randint(-1,1))

pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


# expr = genFull(pset, min_=1, max_=5)
# tree = PrimitiveTree(expr)
# type(tree) == PrimitiveTree




creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", PrimitiveTree, fitness=creator.FitnessMin, pset=pset, batchfitness=[], temp=TEMPERATURE)

toolbox = base.Toolbox()
toolbox.register("expr", genFull, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

# toolbox.register("row", tools.initRepeat, list, toolbox.individual, N_ROWS)
# toolbox.register("population", tools.initRepeat, list, toolbox.row, N_COLUMNS)





# type(individual) = deap.creator.Individual
# inner_grid = np.array(population)

grid = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2), dtype=deap.creator.Individual)


      
def wrapGrid():
    inner_grid = grid[1:-1,1:-1]
    grid[1:-1, 1:-1] = inner_grid
    grid[:1,1:-1] = inner_grid[-1]
    grid[-1:,1:-1] = inner_grid[0]
    grid[1:-1,-1:] = inner_grid[:,:1]
    grid[1:-1,:1] = inner_grid[:,-1:]
    grid[0][0] = inner_grid[-1][-1]
    grid[-1][-1] = inner_grid[0][0]
    grid[0][-1] = inner_grid[-1][0]
    grid[-1][0] = inner_grid[0][-1]



    




toolbox.register("compile", compile, pset=pset)
# pop_grid = wrapGrid(toolbox.population())






# print(x[1][1])
# print(x[9][9])

# tree = gp.PrimitiveTree(expr)

# function = gp.compile(str(x[1][1]), pset)


C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [104]:
def pearsonCheck():
    return random.random()

def fitness_update(indicies_tuple):
    # should simultaneously update batch fitnesses
    return random.randint(1,90)


def checkHood(new_tree, location, temp=TEMPERATURE):
    for i in range(location[0]-1, location[0] + 2):
        for j in range(location[1] - 1, location[1] + 2):
            if (i,j) != tuple(location):
                if type(grid[i][j]) == deap.creator.Individual: # neighbour presence check
                    if pearsonCheck(grid[i][j], new_tree) > float(P_CORRELATION_BOUND * (1/temp)): # pearson check 
                        return 0
    return 1


def initializePOPtoGrid():
    fillq = [[i,j] for i in range(1, N_ROWS + 1) for j in range(1, N_COLUMNS + 1)]
    random.shuffle(fillq)
    
    for location in fillq:
        sapling = generateTree()
        while checkHood(sapling, location) != 1:
            sapling = generateTree()
        grid[i][j] = sapling

        if location[0] in (1, N_ROWS) or location[1] in (1, N_COLUMNS):
            wrapGrid()
            
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            fitness_update(grid[i][j])
            
            
            

In [105]:
def pickFittestNeighbour(location, avoid=[]):
    fittest = (location[0] - 1, location[1] - 1)
    for i in range(location[0]-1, location[0] + 2):
        for j in range(location[1] - 1, location[1] + 2):
            if (i,j) not in avoid and grid[i][j].fitness.values[0] > grid[fittest[0]][fittest[1]].values[0]:
                fittest = (i,j) 
    return fittest




def performCross(location, neighbour):
    children = cxOnePoint(grid[location[0][location[1]]], grid[neighbour[0]][neighbour[1]])
    fitness_update(children[0]); fitness_update(children[1])
    return children[0] if children[0].fitness.values[0] > children[1].fitness.values[0] else children[1]

    

    

operator_matrix = [['c' for i in range(N_ROWS + 1)] for j in range(N_COLUMNS + 1)]
temperature_matrix = [[0.0 for i in range(N_ROWS + 1)] for j in range(N_COLUMNS + 1)]
fitness_matrix = [[0 for i in range(N_ROWS + 1)] for j in range(N_COLUMNS + 1)]



new_gen_pop = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2))
max_fitness = ((0,0), 0.0)
for gen in range(NO_OF_GENERATIONS):
    print('#GEN', gen)
    # GENERATION ROUTINE
    # TODO: EVERY CELL CAN BE THREADED
    for i in range(1, N_ROWS + 1):
        for j in range(1, N_COLUMNS + 1):
            dice = np.random.random()
            # crossover block
            if dice < Pc:
                for trials in range(8):
                    avoid = []
                    neighbour = pickFittestNeighbour((i,j), avoid)
                    child = performCross((i,j), neighbour)
                    # 
                    if (child.fitness.values[0] > grid[i][j].fitness.values[0] and checkHood(child, (i,j))) or np.random.random() < grid[i][j].temp:
                        new_gen_pop[i][j] = child
                        # TODO: similarity check for child passing 
                        operator_matrix[i][j] = 'c'
                        break
                    else:
                        avoid.append(neighbour)
                        continue
                new_gen_pop[i][j] = grid[i][j] # reproduction if no fitter individual is found
                operator_matrix[i][j] = 'cr'
            # mutation block 
            elif dice < Pc + Pm:
                # mutate sub tree then update fitness
                new_gen_pop[i][j] = mutUniform(grid[i][j], expr)[0]
                operator_matrix[i][j] = 'm'
                fitness_update(new_gen_pop[i][j])
            # reproduction block
            else: 
                new_gen_pop[i][j] = grid[i][j]
                operator_matrix[i][j] = 'r'
    print('operator matrix\n', operator_matrix)
    
    # temperature updates
    for i in range(1, N_ROWS + 1):
        for j in range(1, N_COLUMNS + 1):
            alpha = (9.5 + (np.random.random() / 2)) / 10
            grid[i][j].temp = grid[i][j].temp * alpha
            temperature_matrix[i][j] = grid[i][j].temp * alpha
    print('temperature matrix\n', temperature_matrix)
    
    # max fitness search
    # construct fitness matrix for verbosity
    for i in range(1, N_ROWS + 1):
        for j in range(1, N_COLUMNS + 1):
            cursor = new_gen_pop[i][j]
            max_fitness =  ((i,j), cursor.fitness.values[0]) if max_fitness[1] < cursor.fitness.values[0] else max_fitness
            fitness_matrix[i][j] = cursor.fitness.values[0]                
    print('fitness_matrix\n', fitness_matrix)
    print('Gen', gen, 'max fitness', max_fitness)
    if max_fitness > 90:
        print('optimal fitness achieved, halting...')
        break
    
    # reinitialize grids
    grid = new_gen_pop
    wrapGrid()
    new_gen_pop = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2), dtype=deap.creator.Individual)
    

    
    

    





        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

#GEN 0


AttributeError: 'NoneType' object has no attribute 'fitness'